<a href="https://colab.research.google.com/github/katelynnrachel/ckids_fred_morstatter/blob/amykim-analysis/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Get the tweet counts per victim
data = pd.read_csv('./sample_data/df-relatedToHomicide.csv')
data.drop(['Unnamed: 0'], axis=1, inplace=True)
df = data.groupby(['victim_name', 'victim_age', 'victim_gender', 'victim_race', 'cause_of_death', 'if_officer_involved', 'death_location', 'agency'], sort=False).size().reset_index(name='counts')

df['victim_age'] = df['victim_age'].apply(lambda x: np.nan if x == 'None' else x)
df['victim_age'] = df['victim_age'].astype(float)
df['victim_age'].fillna(df['victim_age'].mean(), inplace=True)
df['victim_age'] = df['victim_age'].round().astype(int)

# Create age ranges
age_bins = np.arange(0, 90, 5)
age_labels = [f'{i}-{i+4}' for i in age_bins[:-1]]
df['age_range'] = pd.cut(df['victim_age'], bins=age_bins, labels=age_labels)
df

,victim_name,victim_age,victim_gender,victim_race,cause_of_death,if_officer_involved,death_location,agency,counts,age_range
0,Charles Robert Towns,47,Male,Black,Unknown,Officer-involved,In Front Of 2157 El Serano Ave,LASD,40,45-49
1,Ming Wei Ma,72,Male,Asian,Unknown,Unknown,122 West Garvey Ave #B,LASD,19048,70-74
2,Wen Tau Yu,64,Male,Asian,Unknown,Unknown,122 West Garvey Avenue #B,LASD,208,60-64
3,Valentino Marcos Alvero,68,Male,Asian,Unknown,Unknown,122 West Garvey Avenue #B,LASD,201,65-69
4,Chia Ling Yau,76,Male,Asian,Unknown,Unknown,122 West Garvey Avenue #B,LASD,207,75-79
...,...,...,...,...,...,...,...,...,...,...
469,Michael Di'Shawn Radford,20,Male,Black,Gunshot,Unknown,Torrance,Torrance PD,1,15-19
470,Astin Kyle Edwards,28,Male,Black,Gunshot,Unknown,Torrance,Torrance PD,1,25-29
471,Frank Borsotti,61,Male,White,Blunt force,Unknown,Lancaster,LASD,2,60-64
472,Fael Valente,30,Male,Latino,Gunshot,Unknown,East Los Angeles,LASD,1,25-29


In [3]:
categorical_cols = ['age_range', 'victim_gender', 'victim_race', 'cause_of_death', 'if_officer_involved']
df_encoded = pd.get_dummies(df, columns=categorical_cols)
df_encoded = df_encoded.drop(['cause_of_death_Unknown','victim_race_None', 'victim_race_Unknown'], axis=1)
df_encoded

,victim_name,victim_age,death_location,agency,counts,age_range_0-4,age_range_5-9,age_range_10-14,age_range_15-19,age_range_20-24,...,victim_race_Latino,victim_race_Other,victim_race_White,cause_of_death_Blunt force,cause_of_death_Gunshot,cause_of_death_Other,cause_of_death_Pending,cause_of_death_Stabbing,if_officer_involved_Officer-involved,if_officer_involved_Unknown
0,Charles Robert Towns,47,In Front Of 2157 El Serano Ave,LASD,40,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,Ming Wei Ma,72,122 West Garvey Ave #B,LASD,19048,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Wen Tau Yu,64,122 West Garvey Avenue #B,LASD,208,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Valentino Marcos Alvero,68,122 West Garvey Avenue #B,LASD,201,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,Chia Ling Yau,76,122 West Garvey Avenue #B,LASD,207,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,Michael Di'Shawn Radford,20,Torrance,Torrance PD,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
470,Astin Kyle Edwards,28,Torrance,Torrance PD,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
471,Frank Borsotti,61,Lancaster,LASD,2,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
472,Fael Valente,30,East Los Angeles,LASD,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1


--------------------victim_race------------------

In [12]:
X_race = df_encoded.drop(['victim_name', 'victim_age', 'death_location', 'agency'] + list(df_encoded.filter(regex = 'victim_race')), axis=1)  #features
y_race = df['victim_race']  #target variable

X_train, X_test, y_train, y_test = train_test_split(X_race, y_race, test_size=0.2, random_state=42)

treeModel = DecisionTreeClassifier()
treeModel.fit(X_train, y_train)
print(f'Mean accuracy of race: {treeModel.score(X_test, y_test)}')
print(X_race.columns)

from collections import Counter
print(Counter(y_test))

Mean accuracy of race: 0.43157894736842106
Index(['counts', 'age_range_0-4', 'age_range_5-9', 'age_range_10-14',
       'age_range_15-19', 'age_range_20-24', 'age_range_25-29',
       'age_range_30-34', 'age_range_35-39', 'age_range_40-44',
       'age_range_45-49', 'age_range_50-54', 'age_range_55-59',
       'age_range_60-64', 'age_range_65-69', 'age_range_70-74',
       'age_range_75-79', 'age_range_80-84', 'victim_gender_Female',
       'victim_gender_Male', 'cause_of_death_Blunt force',
       'cause_of_death_Gunshot', 'cause_of_death_Other',
       'cause_of_death_Pending', 'cause_of_death_Stabbing',
       'if_officer_involved_Officer-involved', 'if_officer_involved_Unknown'],
      dtype='object')
Counter({'Latino': 56, 'Black': 29, 'White': 4, 'None': 3, 'Asian': 2, 'Other': 1})


In [5]:
y_race

0       Black
1       Asian
2       Asian
3       Asian
4       Asian
        ...  
469     Black
470     Black
471     White
472    Latino
473    Latino
Name: victim_race, Length: 474, dtype: object

--------------------victim_gender------------------

In [11]:
X_gender = df_encoded.drop(['victim_name', 'victim_age', 'death_location', 'agency'] + list(df_encoded.filter(regex = 'victim_gender')), axis=1) # features
y_gender = df['victim_gender']  # target variable

X_train, X_test, y_train, y_test = train_test_split(X_gender, y_gender, test_size=0.2, random_state=42)

treeModel = DecisionTreeClassifier()
treeModel.fit(X_train, y_train)
print(f'Mean accuracy of gender: {treeModel.score(X_test, y_test)}')
print(X_gender.columns)
print(sum(y_test == 'Male'))
print(len(y_test))

Mean accuracy of gender: 0.8842105263157894
Index(['counts', 'age_range_0-4', 'age_range_5-9', 'age_range_10-14',
       'age_range_15-19', 'age_range_20-24', 'age_range_25-29',
       'age_range_30-34', 'age_range_35-39', 'age_range_40-44',
       'age_range_45-49', 'age_range_50-54', 'age_range_55-59',
       'age_range_60-64', 'age_range_65-69', 'age_range_70-74',
       'age_range_75-79', 'age_range_80-84', 'victim_race_Asian',
       'victim_race_Black', 'victim_race_Korean', 'victim_race_Latino',
       'victim_race_Other', 'victim_race_White', 'cause_of_death_Blunt force',
       'cause_of_death_Gunshot', 'cause_of_death_Other',
       'cause_of_death_Pending', 'cause_of_death_Stabbing',
       'if_officer_involved_Officer-involved', 'if_officer_involved_Unknown'],
      dtype='object')
87
95


--------------------victim_age------------------

In [7]:
X_age = df_encoded.drop(['victim_name', 'victim_age', 'death_location', 'agency'] + list(df_encoded.filter(regex = 'age_range')), axis=1) # features
y_age = df['victim_age']  # target variable
# TODO: decision tree depth, cross validation, hyperparameter, try another model, put code on github
X_train, X_test, y_train, y_test = train_test_split(X_age, y_age, test_size=0.2, random_state=42)

treeModel = DecisionTreeClassifier()
treeModel.fit(X_train, y_train)
print(f'Mean accuracy of age: {treeModel.score(X_test, y_test)}')
print(X_age.columns)

Mean accuracy of age: 0.05263157894736842
Index(['counts', 'victim_gender_Female', 'victim_gender_Male',
       'victim_race_Asian', 'victim_race_Black', 'victim_race_Korean',
       'victim_race_Latino', 'victim_race_Other', 'victim_race_White',
       'cause_of_death_Blunt force', 'cause_of_death_Gunshot',
       'cause_of_death_Other', 'cause_of_death_Pending',
       'cause_of_death_Stabbing', 'if_officer_involved_Officer-involved',
       'if_officer_involved_Unknown'],
      dtype='object')
